In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("..")
import random
from dotmap import DotMap

import numpy as np
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn

from MSMatch.utils import net_builder, get_logger, count_parameters, create_dir_str
from MSMatch.train_utils import TBLog, get_OPT, get_cosine_schedule_with_warmup
from MSMatch.models.fixmatch.FixMatch import FixMatch
from MSMatch.datasets.SSL_Dataset import SSL_Dataset
from MSMatch.datasets.data_utils import get_data_loader

In [ ]:
cfg = DotMap(_dynamic = False)
cfg.dataset = "eurosat_rgb"
cfg.net = "unet"
cfg.batch_size = 32
cfg.p_cutoff = 0.95
cfg.lr = 0.02
cfg.uratio = 7
cfg.weight_decay = 7.5e-4
cfg.ulb_loss_ratio = 1.0
cfg.seed = 42
cfg.num_labels = 100
cfg.opt = "SGD"
cfg.pretrained = False
cfg.save_dir = "./saved_models"
cfg.save_name = "test"
cfg.ema_m = 0.999
cfg.bn_momentum = 1.0 - cfg.ema_m
cfg.eval_batch_size = 1024
cfg.momentum = 0.9
cfg.T = 0.5
cfg.amp = False
cfg.hard_label = True
cfg.multiprocessing_distributed = False
cfg.num_eval_iter = 1000

# random seed has to be set for the syncronization of labeled data sampling in each process.
random.seed(cfg.seed)
torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)
cudnn.deterministic = True

# SET save_path and logger
dir_name = create_dir_str(cfg)
cfg.save_name = os.path.join(cfg.save_name, dir_name)
save_path = os.path.join(cfg.save_dir, cfg.save_name)
tb_log = TBLog(save_path, "")
logger_level = "INFO"
logger = get_logger(cfg.save_name, save_path, logger_level)

In [ ]:
# Construct Dataset
train_dset = SSL_Dataset(
    name=cfg.dataset, train=True, data_dir=None, seed=cfg.seed,
)
lb_dset, ulb_dset = train_dset.get_ssl_dset(cfg.num_labels)

cfg.num_classes = train_dset.num_classes
cfg.num_channels = train_dset.num_channels

_eval_dset = SSL_Dataset(
    name=cfg.dataset, train=False, data_dir=None, seed=cfg.seed,
)
eval_dset = _eval_dset.get_dset()

In [ ]:
_net_builder = net_builder(
    cfg.net,
    pretrained=cfg.pretrained,
    in_channels=cfg.num_channels,
)

In [ ]:
model = FixMatch(
        _net_builder,
        cfg.num_classes,
        cfg.num_channels,
        cfg.ema_m,
        T=0.5,
        p_cutoff=cfg.p_cutoff,
        lambda_u=cfg.ulb_loss_ratio,
        hard_label=True,
        num_eval_iter=cfg.num_eval_iter,
        tb_log=tb_log,
        logger=logger,
    )
logger.info(f"Number of Trainable Params: {count_parameters(model.train_model)}")

In [ ]:
cfg.epoch = 10
cfg.num_train_iter = cfg.epoch * cfg.num_eval_iter * 32 // cfg.batch_size

In [ ]:
optimizer = get_OPT(
    model.train_model, cfg.opt, cfg.lr, cfg.momentum, cfg.weight_decay
)
scheduler = get_cosine_schedule_with_warmup(
    optimizer, cfg.num_train_iter, num_warmup_steps=cfg.num_train_iter * 0
)
model.set_optimizer(optimizer, scheduler)
if torch.cuda.is_available():
    cfg.gpu = 0
    torch.cuda.set_device(cfg.gpu)
    model.train_model = model.train_model.cuda(cfg.gpu)
    model.eval_model = model.eval_model.cuda(cfg.gpu)

logger.info(f"model_arch: {model}")
logger.info(f"Arguments: {cfg}")

In [ ]:
# Construct data loader
loader_dict = {}
dset_dict = {"train_lb": lb_dset, "train_ulb": ulb_dset, "eval": eval_dset}

loader_dict["train_lb"] = get_data_loader(
    dset_dict["train_lb"],
    cfg.batch_size,
    data_sampler="RandomSampler",
    num_iters=cfg.num_train_iter,
    num_workers=1,
    distributed=False,
)

loader_dict["train_ulb"] = get_data_loader(
    dset_dict["train_ulb"],
    cfg.batch_size * cfg.uratio,
    data_sampler="RandomSampler",
    num_iters=cfg.num_train_iter,
    num_workers=4,
    distributed=False,
)

loader_dict["eval"] = get_data_loader(
    dset_dict["eval"], cfg.eval_batch_size, num_workers=1
)

## set DataLoader on FixMatch
model.set_data_loader(loader_dict)

In [ ]:
trainer = model.train
print(cfg)

for epoch in range(cfg.epoch):
    print(epoch)
    trainer(cfg)

In [ ]:
model.save_model("latest_model.pth", save_path)